In [1]:
import pandas as pd
import pandasql as ps

# read data and check

In [2]:
data = pd.read_csv('./data/data.csv')
data.rename(columns={'Month': 'month', 'Sunspots': 'sunspots'}, inplace=True)
data['dates'] = data['month'] + '-01'
data['dates'] = pd.to_datetime(data['dates'])
data.drop(columns=['month'], inplace=True)
data['dates'] = data['dates'] + pd.offsets.MonthEnd(0)
data

,sunspots,dates
0,58.0,1749-01-31
1,62.6,1749-02-28
2,70.0,1749-03-31
3,55.7,1749-04-30
4,85.0,1749-05-31
...,...,...
2815,71.8,1983-08-31
2816,50.3,1983-09-30
2817,55.8,1983-10-31
2818,33.3,1983-11-30


In [3]:
# data has no duplicates
data[data.duplicated(subset=['dates'], keep=False)]

,sunspots,dates


In [4]:
data.dates.min(), data.dates.max()

(Timestamp('1749-01-31 00:00:00'), Timestamp('1983-12-31 00:00:00'))

In [5]:
# value is na?
data.isna().sum()

sunspots    0
dates       0
dtype: int64

In [6]:
data.shape

(2820, 2)

# feature engineering

In [7]:
# month, year
f_data = data.copy(deep=True)
f_data['year'] = f_data['dates'].dt.year
f_data['month'] = f_data['dates'].dt.month
f_data

,sunspots,dates,year,month
0,58.0,1749-01-31,1749,1
1,62.6,1749-02-28,1749,2
2,70.0,1749-03-31,1749,3
3,55.7,1749-04-30,1749,4
4,85.0,1749-05-31,1749,5
...,...,...,...,...
2815,71.8,1983-08-31,1983,8
2816,50.3,1983-09-30,1983,9
2817,55.8,1983-10-31,1983,10
2818,33.3,1983-11-30,1983,11


In [8]:
# lag
sql = f"""select *,
    LAG(sunspots, 1) OVER(ORDER BY year, month) AS lag_1_sunspots,
    LAG(sunspots, 2) OVER(ORDER BY year, month) AS lag_2_sunspots,
    LAG(sunspots, 3) OVER(ORDER BY year, month) AS lag_3_sunspots,
    LAG(sunspots, 4) OVER(ORDER BY year, month) AS lag_4_sunspots,
    LAG(sunspots, 5) OVER(ORDER BY year, month) AS lag_5_sunspots,
    LAG(sunspots, 6) OVER(ORDER BY year, month) AS lag_6_sunspots,
    LAG(sunspots, 12) OVER(ORDER BY year, month) AS lag_12_sunspots,
    LAG(sunspots, 24) OVER(ORDER BY year, month) AS lag_24_sunspots,
    LAG(sunspots, 36) OVER(ORDER BY year, month) AS lag_36_sunspots
from f_data
"""
f_data = ps.sqldf(sql, locals())
f_data

,sunspots,dates,year,month,lag_1_sunspots,lag_2_sunspots,lag_3_sunspots,lag_4_sunspots,lag_5_sunspots,lag_6_sunspots,lag_12_sunspots,lag_24_sunspots,lag_36_sunspots
0,58.0,1749-01-31 00:00:00.000000,1749,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62.6,1749-02-28 00:00:00.000000,1749,2,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70.0,1749-03-31 00:00:00.000000,1749,3,62.6,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55.7,1749-04-30 00:00:00.000000,1749,4,70.0,62.6,58.0,NaN,NaN,NaN,NaN,NaN,NaN
4,85.0,1749-05-31 00:00:00.000000,1749,5,55.7,70.0,62.6,58.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,71.8,1983-08-31 00:00:00.000000,1983,8,82.2,91.1,99.2,80.7,66.5,51.0,107.6,158.7,135.4
2816,50.3,1983-09-30 00:00:00.000000,1983,9,71.8,82.2,91.1,99.2,80.7,66.5,118.8,167.3,155.0
2817,55.8,1983-10-31 00:00:00.000000,1983,10,50.3,71.8,82.2,91.1,99.2,80.7,94.7,162.4,164.7
2818,33.3,1983-11-30 00:00:00.000000,1983,11,55.8,50.3,71.8,82.2,91.1,99.2,98.1,137.5,147.9


In [9]:
# roll_avg
sql = f"""select *,
    avg(sunspots) OVER(ORDER BY year, month rows between 2 preceding and current row) AS avg_3_sunspots,
    avg(sunspots) OVER(ORDER BY year, month rows between 5 preceding and current row) AS avg_6_sunspots,
    avg(sunspots) OVER(ORDER BY year, month rows between 11 preceding and current row) AS avg_12_sunspots,
    avg(sunspots) OVER(ORDER BY year, month rows between 23 preceding and current row) AS avg_24_sunspots,
    avg(sunspots) OVER(ORDER BY year, month rows between 35 preceding and current row) AS avg_36_sunspots
from f_data
"""
f_data = ps.sqldf(sql, locals())
f_data

,sunspots,dates,year,month,lag_1_sunspots,lag_2_sunspots,lag_3_sunspots,lag_4_sunspots,lag_5_sunspots,lag_6_sunspots,lag_12_sunspots,lag_24_sunspots,lag_36_sunspots,avg_3_sunspots,avg_6_sunspots,avg_12_sunspots,avg_24_sunspots,avg_36_sunspots
0,58.0,1749-01-31 00:00:00.000000,1749,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.000000,58.000000,58.000000,58.000000,58.000000
1,62.6,1749-02-28 00:00:00.000000,1749,2,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.300000,60.300000,60.300000,60.300000,60.300000
2,70.0,1749-03-31 00:00:00.000000,1749,3,62.6,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.533333,63.533333,63.533333,63.533333,63.533333
3,55.7,1749-04-30 00:00:00.000000,1749,4,70.0,62.6,58.0,NaN,NaN,NaN,NaN,NaN,NaN,62.766667,61.575000,61.575000,61.575000,61.575000
4,85.0,1749-05-31 00:00:00.000000,1749,5,55.7,70.0,62.6,58.0,NaN,NaN,NaN,NaN,NaN,70.233333,66.260000,66.260000,66.260000,66.260000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,71.8,1983-08-31 00:00:00.000000,1983,8,82.2,91.1,99.2,80.7,66.5,51.0,107.6,158.7,135.4,81.700000,81.916667,88.783333,109.983333,120.800000
2816,50.3,1983-09-30 00:00:00.000000,1983,9,71.8,82.2,91.1,99.2,80.7,66.5,118.8,167.3,155.0,68.100000,79.216667,83.075000,105.108333,117.891667
2817,55.8,1983-10-31 00:00:00.000000,1983,10,50.3,71.8,82.2,91.1,99.2,80.7,94.7,162.4,164.7,59.300000,75.066667,79.833333,100.666667,114.866667
2818,33.3,1983-11-30 00:00:00.000000,1983,11,55.8,50.3,71.8,82.2,91.1,99.2,98.1,137.5,147.9,46.466667,64.083333,74.433333,96.325000,111.683333


In [10]:
# his_avg/min/max
sql = f"""select *,
    max(sunspots) OVER(ORDER BY year, month rows between unbounded preceding and current row) AS roll_max_sunspots,
    min(sunspots) OVER(ORDER BY year, month rows between unbounded preceding and current row) AS roll_min_sunspots,
    avg(sunspots) OVER(ORDER BY year, month rows between unbounded preceding and current row) AS roll_avg_sunspots
from f_data
"""
f_data = ps.sqldf(sql, locals())
f_data

,sunspots,dates,year,month,lag_1_sunspots,lag_2_sunspots,lag_3_sunspots,lag_4_sunspots,lag_5_sunspots,lag_6_sunspots,...,lag_24_sunspots,lag_36_sunspots,avg_3_sunspots,avg_6_sunspots,avg_12_sunspots,avg_24_sunspots,avg_36_sunspots,roll_max_sunspots,roll_min_sunspots,roll_avg_sunspots
0,58.0,1749-01-31 00:00:00.000000,1749,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,58.0,58.000000
1,62.6,1749-02-28 00:00:00.000000,1749,2,58.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,60.300000,60.300000,60.300000,60.300000,60.300000,62.6,58.0,60.300000
2,70.0,1749-03-31 00:00:00.000000,1749,3,62.6,58.0,NaN,NaN,NaN,NaN,...,NaN,NaN,63.533333,63.533333,63.533333,63.533333,63.533333,70.0,58.0,63.533333
3,55.7,1749-04-30 00:00:00.000000,1749,4,70.0,62.6,58.0,NaN,NaN,NaN,...,NaN,NaN,62.766667,61.575000,61.575000,61.575000,61.575000,70.0,55.7,61.575000
4,85.0,1749-05-31 00:00:00.000000,1749,5,55.7,70.0,62.6,58.0,NaN,NaN,...,NaN,NaN,70.233333,66.260000,66.260000,66.260000,66.260000,85.0,55.7,66.260000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,71.8,1983-08-31 00:00:00.000000,1983,8,82.2,91.1,99.2,80.7,66.5,51.0,...,158.7,135.4,81.700000,81.916667,88.783333,109.983333,120.800000,253.8,0.0,51.277415
2816,50.3,1983-09-30 00:00:00.000000,1983,9,71.8,82.2,91.1,99.2,80.7,66.5,...,167.3,155.0,68.100000,79.216667,83.075000,105.108333,117.891667,253.8,0.0,51.277068
2817,55.8,1983-10-31 00:00:00.000000,1983,10,50.3,71.8,82.2,91.1,99.2,80.7,...,162.4,164.7,59.300000,75.066667,79.833333,100.666667,114.866667,253.8,0.0,51.278673
2818,33.3,1983-11-30 00:00:00.000000,1983,11,55.8,50.3,71.8,82.2,91.1,99.2,...,137.5,147.9,46.466667,64.083333,74.433333,96.325000,111.683333,253.8,0.0,51.272295


In [11]:
f_data['dates'] = pd.to_datetime(f_data['dates'])
f_data:pd.DataFrame

In [12]:
f_data.dropna(subset=['lag_36_sunspots'], inplace=True)
f_data.shape

(2784, 21)

# make y

In [13]:
# m + 120 - m + 132
# 12 model
# 12 y

In [15]:
for i in range(120, 133):
    print(i)
    final_data: pd.DataFrame = f_data.copy(deep=True)
    final_data['target_dates'] = final_data['dates'] + pd.DateOffset(months=i) + pd.offsets.MonthEnd(0) # + 120 month
    y = final_data[['dates', 'sunspots']].copy(deep=True)
    y.rename(columns={'sunspots': 'target_sunspots', 'dates': 'target_dates'}, inplace=True)
    final_data = final_data.merge(y, on='target_dates', how='left')
    final_data.dropna(subset=['target_sunspots'], inplace=True)
    print(final_data.shape)
    final_data.to_csv(f'./data/processed_data_{i}.csv', index=False)

120
(2664, 23)
121
(2663, 23)
122
(2662, 23)
123
(2661, 23)
124
(2660, 23)
125
(2659, 23)
126
(2658, 23)
127
(2657, 23)
128
(2656, 23)
129
(2655, 23)
130
(2654, 23)
131
(2653, 23)
132
(2652, 23)
